In [1]:
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, START, END
import requests, os, base64
from sentence_transformers import SentenceTransformer, util
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_tavily import TavilySearch
from newsapi import NewsApiClient
from dotenv import load_dotenv
from tools import serper_search
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
import faiss
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings



load_dotenv()

NEWS_API_KEY = os.environ.get('NEWS_API_KEY')
FREEPIK_API_KEY = os.environ.get('FREEPIK_API_KEY')
TAVILY_API_KEY = os.environ.get('TAVILY_API_KEY')

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
llm = ChatGoogleGenerativeAI( model="gemini-2.0-flash", google_api_key=os.environ["GEMINI_API_KEY"])
# search_tool = TavilySearch(max_results=3,topic="news",search_depth="advanced")
search_tool = serper_search()
newsapi = NewsApiClient(api_key=NEWS_API_KEY)

VECTOR_DB_PATH = "vector_store"
if os.path.exists(VECTOR_DB_PATH):
    db = FAISS.load_local(
    folder_path=VECTOR_DB_PATH,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

else:
    dimension = len(embedding_model.embed_query("test")) 
    index = faiss.IndexFlatL2(dimension)
    docstore = InMemoryDocstore()
    db = FAISS(embedding_model, index, docstore, {})



class AgentState(TypedDict):
    user_search_request: str
    query_type: str
    search_queries: List[str]
    summarize_queries: List[dict]
    final_report: List[dict]
    prompt: str
    img_data: List[dict]

def classify_query(state: AgentState) -> AgentState:
    prompt = f"""
    Classify the following query into one of the following types:
    - factual
    - trending_news
    - opinion
    - comparison
    - technical

    Only return the type with no punctuation or explanation.

    Query: {state['user_search_request']}
    """
    response = llm.invoke(prompt)
    state['query_type'] = response.content.strip().lower()
    return state


def split_topic(state: AgentState) -> AgentState:
    try:
        topic = state['user_search_request']
        split_prompt = f"""
        Break the following topic into 3 to 5 distinct sub-questions. One question per line, no numbering.
        Topic: {topic}
        """
        result = llm.invoke(split_prompt)
        subquestions = [line.strip() for line in result.content.splitlines() if line.strip()]
        state['search_queries'] = subquestions
    except Exception as e:
        print(f"split_topic error: {e}")
        state['search_queries'] = [state['user_search_request']]
    return state


def split_topic_or_not(state: AgentState) -> str:
    if state['query_type'] in ['factual', 'technical', 'comparison']:
        return "split"
    else:
        state['search_queries'] = [state['user_search_request']]
        return "no_split"


def inject_original_query(state: AgentState) -> AgentState:
    state['search_queries'] = [state['user_search_request']]
    return state


def search_node(state: AgentState) -> AgentState:
    responses = []

    for query in state['search_queries']:
        results = fetch_web_and_news_results(query, state['query_type'])
        deduped = deduplicate_sources(results)
        docs = [
            Document(page_content=src["content"], metadata={"source": src["url"]})
            for src in deduped if src.get("content")
        ]
        db.add_documents(docs)
        responses.append({"query": query, "results": deduped})

    db.save_local(VECTOR_DB_PATH)
    state['summarize_queries'] = responses
    return state


def fetch_web_and_news_results(query: str, query_type: str) -> List[dict]:

    search_results = search_tool.invoke(query)
    url = [article['link'] for article in search_results.get('organic',[])] 

    for article in search_results.get('organic',[]):
        print(article['link'])


    results = []
    try:
        web = search_tool.invoke(query)
        if web.get("results"):
            results.extend(web["results"])
    except Exception as e:
        print(f"Search tool failed: {e}")

    
    if query_type == "trending_news":
        try:
            news = newsapi.get_everything(q=query, page_size=5, sort_by="publishedAt")
            for article in news.get("articles", []):
                results.append({
                    "url": article.get("url"),
                    "content": article.get("description") or article.get("content"),
                    "source": article.get("source", {}).get("name", "NewsAPI")
                })
        except Exception as e:
            print(f"NewsAPI error: {e}")

    return results



def deduplicate_sources(sources: List[dict], threshold=0.85) -> List[dict]:
    unique = []
    embeddings = []
    for src in sources:
        text = src.get("content", "")
        if not text:
            continue
        emb = sentence_model.encode(text, convert_to_tensor=True)
        if all(util.cos_sim(emb, e).item() < threshold for e in embeddings):
            embeddings.append(emb)
            unique.append(src)
    return unique



def summarize_node(state: AgentState) -> AgentState:
    summary = []
    for item in state['summarize_queries']:
        question = item['query']
        retrieved_docs = db.similarity_search(question, k=4)
        sources = item.get('results', [])
        if not retrieved_docs:
            continue

        source_text = "\n".join(
            f"[{i+1}] {doc.page_content} ({doc.metadata.get('source', 'unknown')})"
            for i, doc in enumerate(retrieved_docs)
        )

        sum_prompt = f"""
        You are a journalist writing for CoinEdition. Maintain a professional tone, concise structure, and use inline citations like [1], [2].
        
        Write a well-written news-style summary of the following.
        
        Question: {question}
        
        Sources:
        {source_text}
        
        Answer:
        """
        response = llm.invoke(sum_prompt)
        summary.append({"question": question, "summary": response.content.strip()})
    state['final_report'] = summary
    return state



def img_prompter(state: AgentState) -> AgentState:
    context = "\n\n".join([s["summary"] for s in state["final_report"]])
    prompt = f"""
    You are a professional AI image prompt writer. Based on the content below, create a vivid, cinematic-style visual prompt suitable for AI models like Midjourney, Stable Diffusion, or Flux.

    Focus on realistic, detailed, and visually compelling imagery — include setting, characters (if any), lighting, mood, atmosphere, and color scheme.

    Avoid mentioning the source content. Return only the generated image prompt as a single line.

    Content:
    {context}
    """
    response = llm.invoke(prompt)
    state["prompt"] = response.content.strip()
    return state



def img_generator(state: AgentState) -> AgentState:
    url = "https://api.freepik.com/v1/ai/text-to-image"
    req = {
        "prompt": state['prompt'],
        "seed": 42,
        "num_images": 1,
        "image": {"size": "square_1_1"},
    }
    headers = {
        "x-freepik-api-key": FREEPIK_API_KEY,
        "Content-Type": "application/json"
    }
    response = requests.post(url, json=req, headers=headers)
    if response.status_code == 200:
        data = response.json()
        base64_image = data.get("data", [{}])[0].get("base64")
        if base64_image:
            image_path = "generated_image.jpg"
            with open(image_path, "wb") as f:
                f.write(base64.b64decode(base64_image))
            state['img_data'] = [{"file_path": os.path.abspath(image_path), "status": "saved"}]
        else:
            state['img_data'] = [{"status": "no_base64_returned"}]
    else:
        state['img_data'] = [{"status": "error", "code": response.status_code, "message": response.text}]
    return state




graph = StateGraph(AgentState)


graph.add_node("classify_query", classify_query)
graph.add_node("split_topic", split_topic)
graph.add_node("inject_query", inject_original_query)
graph.add_node("search_node", search_node)
graph.add_node("summarize_node", summarize_node)
graph.add_node("img_prompter", img_prompter)
graph.add_node("img_generator", img_generator)


graph.add_edge(START, "classify_query")
graph.add_conditional_edges("classify_query", split_topic_or_not, {
    "split": "split_topic",
    "no_split": "inject_query"
})
graph.add_edge("split_topic", "search_node")
graph.add_edge("inject_query", "search_node")
graph.add_edge("search_node", "summarize_node")
graph.add_edge("summarize_node", "img_prompter")
graph.add_edge("img_prompter", "img_generator")
graph.add_edge("img_generator", END)



graph.compile()
compiled_graph = graph.compile()
initial_state = {
    "user_search_request": "Elon Musk latest tweet",
    "query_type": "",
    "search_queries": [],
    "summarize_queries": [],
    "final_report": [],
    "prompt": "",
    "img_data": []
}

result = compiled_graph.invoke(initial_state)


In [ ]:
from IPython.display import Image, display
display(Image(compiled_graph.get_graph().draw_mermaid_png()))

In [3]:
print(result)

{'user_search_request': 'Elon Musk latest tweet', 'query_type': 'trending_news', 'search_queries': ['Elon Musk latest tweet'], 'summarize_queries': [{'query': 'Elon Musk latest tweet', 'results': [{'url': 'https://www.aljazeera.com/economy/2025/7/11/as-millions-adopt-grok-to-fact-check-misinformation-abounds', 'content': 'As Grok provides instant fact checks to users on X, its ability to make things up is outstripping its usefulness.', 'source': 'Al Jazeera English'}, {'url': 'https://www.carolinecrampton.com/im-done-with-social-media/', 'content': 'Article URL: https://www.carolinecrampton.com/im-done-with-social-media/\nComments URL: https://news.ycombinator.com/item?id=44532913\nPoints: 19\n# Comments: 3', 'source': 'Carolinecrampton.com'}, {'url': 'https://www.counterpunch.org/2025/07/11/an-anarchist-appeal-to-the-disgruntled-deplorable/', 'content': '“Ooh, that smell Can’t you smell that smell? Ooh, that smell The smell of death surrounds you” ‘Ooh That Smell’ by Lynyrd Skynyrd I 